In [15]:
import os
import time
import shutil
import tarfile
import pathlib

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import seaborn as sns
import xgboost as xgb 
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import skew
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error, r2_score
from sklearn.linear_model import Lasso, LassoCV, MultiTaskLassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import optimizers
from keras.initializers import lecun_normal
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.neural_network import MLPRegressor

In [16]:
tf.__version__

'2.1.0'

In [17]:
# CSV TO DATA
def csvToData(csv_file):
    df = pd.read_csv(csv_file, header = 0)
    column = list(df.columns.values)
    data = df.values
    return column, data

sample_column, sample_csv_data = csvToData("data/sample_submission.csv")
test_column, test_csv_data = csvToData("data/test.csv")
train_column, train_csv_data = csvToData("data/train.csv")
print('sample_submission-----------')
print(sample_column)
print(sample_csv_data.shape)
print('test-----------')
print(test_column)
print(test_csv_data.shape)
print('train-----------')
print(train_column)
print(train_csv_data.shape)


sample_submission-----------
['id', 'hhb', 'hbo2', 'ca', 'na']
(10000, 5)
test-----------
['id', 'rho', '650_src', '660_src', '670_src', '680_src', '690_src', '700_src', '710_src', '720_src', '730_src', '740_src', '750_src', '760_src', '770_src', '780_src', '790_src', '800_src', '810_src', '820_src', '830_src', '840_src', '850_src', '860_src', '870_src', '880_src', '890_src', '900_src', '910_src', '920_src', '930_src', '940_src', '950_src', '960_src', '970_src', '980_src', '990_src', '650_dst', '660_dst', '670_dst', '680_dst', '690_dst', '700_dst', '710_dst', '720_dst', '730_dst', '740_dst', '750_dst', '760_dst', '770_dst', '780_dst', '790_dst', '800_dst', '810_dst', '820_dst', '830_dst', '840_dst', '850_dst', '860_dst', '870_dst', '880_dst', '890_dst', '900_dst', '910_dst', '920_dst', '930_dst', '940_dst', '950_dst', '960_dst', '970_dst', '980_dst', '990_dst']
(10000, 72)
train-----------
['id', 'rho', '650_src', '660_src', '670_src', '680_src', '690_src', '700_src', '710_src', '720_s

In [18]:
# Data to DataFrame
sample_data = pd.DataFrame(sample_csv_data, columns=sample_column)
test_data = pd.DataFrame(test_csv_data, columns=test_column)
train_data = pd.DataFrame(train_csv_data, columns=train_column)
print(sample_data.shape, test_data.shape, train_data.shape)

(10000, 5) (10000, 72) (10000, 76)


In [19]:
src_list=['650_src', '660_src', '670_src', '680_src', '690_src', '700_src', '710_src', '720_src', '730_src', 
          '740_src', '750_src', '760_src', '770_src', '780_src', '790_src', '800_src', '810_src', '820_src', 
          '830_src', '840_src', '850_src', '860_src', '870_src', '880_src', '890_src', '900_src', '910_src', 
          '920_src', '930_src', '940_src', '950_src', '960_src', '970_src', '980_src', '990_src']
dst_list=['650_dst', '660_dst', '670_dst', '680_dst', '690_dst', '700_dst', '710_dst', '720_dst', '730_dst', 
          '740_dst', '750_dst', '760_dst', '770_dst', '780_dst', '790_dst', '800_dst', '810_dst', '820_dst', 
          '830_dst', '840_dst', '850_dst', '860_dst', '870_dst', '880_dst', '890_dst', '900_dst', '910_dst', 
          '920_dst', '930_dst', '940_dst', '950_dst', '960_dst', '970_dst', '980_dst', '990_dst']

In [20]:
# 보간법 tqdm , 
from tqdm import tqdm
alpha=train_data[dst_list]
beta=test_data[dst_list]

for i in tqdm(train_data.index):
    alpha.loc[i] = alpha.loc[i].interpolate()
    
for i in tqdm(test_data.index):
    beta.loc[i] = beta.loc[i].interpolate()
    
#위에 보간법을 사용하게되면, 앞단에 비는 경우가 발생. 빌 경우 뒷단에 있는 데이터를 끌어다가 씀.
alpha.loc[alpha['700_dst'].isnull(),'700_dst']=alpha.loc[alpha['700_dst'].isnull(),'710_dst']
alpha.loc[alpha['690_dst'].isnull(),'690_dst']=alpha.loc[alpha['690_dst'].isnull(),'700_dst']
alpha.loc[alpha['680_dst'].isnull(),'680_dst']=alpha.loc[alpha['680_dst'].isnull(),'690_dst']
alpha.loc[alpha['670_dst'].isnull(),'670_dst']=alpha.loc[alpha['670_dst'].isnull(),'680_dst']
alpha.loc[alpha['660_dst'].isnull(),'660_dst']=alpha.loc[alpha['660_dst'].isnull(),'670_dst']
alpha.loc[alpha['650_dst'].isnull(),'650_dst']=alpha.loc[alpha['650_dst'].isnull(),'660_dst']

beta.loc[beta['700_dst'].isnull(),'700_dst']=beta.loc[beta['700_dst'].isnull(),'710_dst']
beta.loc[beta['690_dst'].isnull(),'690_dst']=beta.loc[beta['690_dst'].isnull(),'700_dst']
beta.loc[beta['680_dst'].isnull(),'680_dst']=beta.loc[beta['680_dst'].isnull(),'690_dst']
beta.loc[beta['670_dst'].isnull(),'670_dst']=beta.loc[beta['670_dst'].isnull(),'680_dst']
beta.loc[beta['660_dst'].isnull(),'660_dst']=beta.loc[beta['660_dst'].isnull(),'670_dst']
beta.loc[beta['650_dst'].isnull(),'650_dst']=beta.loc[beta['650_dst'].isnull(),'660_dst']

train_data[dst_list] = np.array(alpha)
test_data[dst_list] = np.array(beta)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2201.66it/s]


In [21]:
# rho 값의 제곱을 dst에 곱해줌.
for col in dst_list:
    train_data[col] = train_data[col] * (train_data['rho'] ** 2)
    test_data[col] = test_data[col] * (test_data['rho']**2)

In [22]:
#원래 빛과 측정빛의 차이값을 변수로 추가
gap_feature_names=[]
for i in range(650, 1000, 10):
    gap_feature_names.append(str(i) + '_gap')

alpha=pd.DataFrame(np.array(train_data[src_list]) - np.array(train_data[dst_list]), columns=gap_feature_names, index=train_data.index)
beta=pd.DataFrame(np.array(test_data[src_list]) - np.array(test_data[dst_list]), columns=gap_feature_names, index=test_data.index)

train_data=pd.concat((train_data, alpha), axis=1)
test_data=pd.concat((test_data, beta), axis=1)

In [23]:
#원래 빛과 측정빛의 차이 비율을 변수 추가
epsilon=1e-10

for dst_col, src_col in zip(dst_list, src_list):
    dst_val=train_data[dst_col]
    src_val=train_data[src_col] + epsilon
    
    delta_ratio = dst_val / src_val
    train_data[dst_col + '_' + src_col + '_ratio'] = delta_ratio
    
    dst_val=test_data[dst_col]
    src_val=test_data[src_col] + epsilon
    
    delta_ratio = dst_val / src_val
    test_data[dst_col + '_' + src_col + '_ratio'] = delta_ratio

In [24]:
##측정 빛에 이산 푸리에 변환
alpha_real=train_data[dst_list]
alpha_imag=train_data[dst_list]

beta_real=test_data[dst_list]
beta_imag=test_data[dst_list]

for i in tqdm(alpha_real.index):
    alpha_real.loc[i]=alpha_real.loc[i] - alpha_real.loc[i].mean()
    alpha_imag.loc[i]=alpha_imag.loc[i] - alpha_real.loc[i].mean()
    
    alpha_real.loc[i] = np.fft.fft(alpha_real.loc[i], norm='ortho').real
    alpha_imag.loc[i] = np.fft.fft(alpha_imag.loc[i], norm='ortho').imag

    
for i in tqdm(beta_real.index):
    beta_real.loc[i]=beta_real.loc[i] - beta_real.loc[i].mean()
    beta_imag.loc[i]=beta_imag.loc[i] - beta_imag.loc[i].mean()
    
    beta_real.loc[i] = np.fft.fft(beta_real.loc[i], norm='ortho').real
    beta_imag.loc[i] = np.fft.fft(beta_imag.loc[i], norm='ortho').imag
    
real_part=[]
imag_part=[]

for col in dst_list:
    real_part.append(col + '_fft_real')
    imag_part.append(col + '_fft_imag')
    
alpha_real.columns=real_part
alpha_imag.columns=imag_part
alpha = pd.concat((alpha_real, alpha_imag), axis=1)

beta_real.columns=real_part
beta_imag.columns=imag_part
beta=pd.concat((beta_real, beta_imag), axis=1)

train_data=pd.concat((train_data, alpha), axis=1)
test_data=pd.concat((test_data, beta), axis=1)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:13<00:00, 750.36it/s]


In [25]:
print(np.isnan(train_data).values.any(),np.isnan(test_data).values.any())

False False


## Data split

training data ( 8000개 ) = train_dataset

validataion data ( 2000개 ) = test_dataset

In [26]:
num_traindata = 8000
num_validdata = 2000
from sklearn.utils import shuffle
train_data = shuffle(train_data)
train_dataset = train_data[:num_traindata]
valid_dataset = train_data[num_traindata : num_traindata + num_validdata]
train_dataset = train_dataset.drop('id',axis = 1)
valid_dataset = valid_dataset.drop('id',axis = 1)
print(train_dataset.shape, valid_dataset.shape)

(8000, 215) (2000, 215)


## Data split

--training data ( 8000 )

x_train

y_train

--validation data ( 2000 )

x_test

y_test

--training data ( 10000 )

x_train_sample 

y_train_sample

In [27]:
#x_train, y_train 분류
x_train = train_dataset.loc[:, 'rho':] # train dataset
x_train.drop(['hhb','hbo2','ca','na'], axis='columns', inplace=True)
y_train = train_dataset.loc[:, 'hhb':'na'] # train labels
x_train.shape, y_train.shape

((8000, 211), (8000, 4))

In [28]:
#x_test, y_test 분류
x_test = valid_dataset.loc[:, 'rho':] # test dataset
x_test.drop(['hhb','hbo2','ca','na'], axis='columns', inplace=True)
y_test = valid_dataset.loc[:, 'hhb':'na'] # test labels
x_test.shape, y_test.shape

((2000, 211), (2000, 4))

In [29]:
#sample Code 용
x_train_sample = train_data.loc[:, 'rho':] # train dataset
x_train_sample.drop(['hhb','hbo2','ca','na'], axis='columns', inplace=True)
y_train_sample = train_data.loc[:, 'hhb':'na'] # train labels
x_train_sample.shape, y_train_sample.shape

((10000, 211), (10000, 4))

In [30]:
#Sample_code
avr_valid_mse = 0
def test_train_model(x_data, y_data, k=5):
    models = []
    
    k_fold = KFold(n_splits=k, shuffle=True, random_state=123)
    
    for train_idx, val_idx in k_fold.split(x_data):
        
        x_train, y_train = x_data.iloc[train_idx], y_data.iloc[train_idx]
        x_val, y_val = x_data.iloc[val_idx], y_data.iloc[val_idx]
    
        d_train = xgb.DMatrix(data = x_train, label = y_train)
        d_val = xgb.DMatrix(data = x_val, label = y_val)
        
        wlist = [(d_train, 'train'), (d_val, 'eval')]
        
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'mae',
            'seed':777
            }


        model = xgb.train(params=params, dtrain=d_train, num_boost_round=1000, verbose_eval=1000, evals=wlist)
        models.append(model)
        
        

    return models

models = {}
for label in y_train.columns:
    print(label)
    models[label] = test_train_model(x_train_sample, y_train_sample[label])
    
    

hhb
[0]	train-mae:5.29064	eval-mae:5.27029
[999]	train-mae:0.000595	eval-mae:0.866566
[0]	train-mae:5.30938	eval-mae:5.19686
[999]	train-mae:0.000609	eval-mae:0.829306
[0]	train-mae:5.29878	eval-mae:5.23914
[999]	train-mae:0.000602	eval-mae:0.866656
[0]	train-mae:5.27926	eval-mae:5.3544
[999]	train-mae:0.00063	eval-mae:0.879947
[0]	train-mae:5.26385	eval-mae:5.4172
[999]	train-mae:0.000625	eval-mae:0.887416
hbo2
[0]	train-mae:2.45828	eval-mae:2.49552
[999]	train-mae:0.00064	eval-mae:0.596674
[0]	train-mae:2.47092	eval-mae:2.42735
[999]	train-mae:0.000637	eval-mae:0.584332
[0]	train-mae:2.46294	eval-mae:2.47591
[999]	train-mae:0.00063	eval-mae:0.590059
[0]	train-mae:2.45718	eval-mae:2.49673
[999]	train-mae:0.000614	eval-mae:0.586745
[0]	train-mae:2.46664	eval-mae:2.43355
[999]	train-mae:0.000616	eval-mae:0.585503
ca
[0]	train-mae:5.99222	eval-mae:6.08401
[999]	train-mae:0.000642	eval-mae:1.86966
[0]	train-mae:6.00984	eval-mae:5.98616
[999]	train-mae:0.000617	eval-mae:1.91088
[0]	train-m

In [31]:
#결과 출력
for col in models:
    print(col)
    preds = []
    for model in models[col]:
        preds.append(model.predict(xgb.DMatrix(test_data.loc[:, 'rho':])))
    pred = np.mean(preds, axis=0)

    sample_data[col] = pred

hhb
hbo2
ca
na


In [32]:
sample_data.head()

,id,hhb,hbo2,ca,na
0,10000,9.018572,4.452666,8.005127,2.846764
1,10001,7.164327,2.323943,9.818594,2.752697
2,10002,9.284598,4.968606,11.628729,3.780931
3,10003,8.377440,4.102247,9.838860,4.921188
4,10004,7.568151,3.207137,8.396446,4.091807


In [ ]:
#Save Excel 
sample_data.to_csv('Dacon_Geonho_20200616_1.csv', index=False)